In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),  # Output a single value for binary classification
    nn.Sigmoid()  # Sigmoid activation for binary classification
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/Users/wmwong/miniforge3/envs/threatsensor/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/wmwong/miniforge3/envs/threatsensor/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import torch.optim as optim

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [3]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class ViolenceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Path to the root directory containing 'violence' and 'non_violence' folders.
            transform (callable, optional): Optional transform to be applied to the images.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Define class labels
        self.classes = ["non_violence", "violence"]
        self.class_to_label = {class_name: idx for idx, class_name in enumerate(self.classes)}

        # Load all images and labels
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                continue  # Skip if the folder doesn't exist

            # Iterate through 'train' and 'val' subfolders
            for split in ["train", "val"]:
                split_dir = os.path.join(class_dir, split)
                if not os.path.exists(split_dir):
                    continue  # Skip if the subfolder doesn't exist

                for img_name in os.listdir(split_dir):
                    img_path = os.path.join(split_dir, img_name)
                    label = self.class_to_label[class_name]
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

In [4]:
# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ViolenceDataset(root_dir="./data/train_val", transform=train_transform)
val_dataset = ViolenceDataset(root_dir="./data/train_val", transform=val_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import recall_score, f1_score

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    all_labels = []
    all_preds = []

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        outputs = outputs.squeeze()  # Remove extra dimensions
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        preds = torch.round(outputs)  # Round predictions to 0 or 1
        running_corrects += torch.sum(preds == labels.data)

        # Store predictions and labels for recall/F1 calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.detach().cpu().numpy())  # Detach before converting to numpy

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    epoch_recall = recall_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, Recall: {epoch_recall:.4f}, F1: {epoch_f1:.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            preds = torch.round(outputs)
            val_corrects += torch.sum(preds == labels.data)

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())  # No need to detach in no_grad() context

        val_loss = val_loss / len(val_dataset)
        val_acc = val_corrects.double() / len(val_dataset)
        val_recall = recall_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds)

        print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

Epoch 1/10, Loss: 0.0073, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0026, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 2/10, Loss: 0.0031, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0015, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 3/10, Loss: 0.0051, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0016, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 4/10, Loss: 0.0028, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0007, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 5/10, Loss: 0.0012, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0007, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 6/10, Loss: 0.0013, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0005, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 7/10, Loss: 0.0010, Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Validation Loss: 0.0006, Validation Acc: 1.0000, Recall: 1.0000, F1: 1.0000
Epoch 

In [9]:
torch.save(model.state_dict(), "../model/model.pt")

In [10]:
def show_misclassified_examples(misclassified_examples, num_examples=3):
    """
    Display a few misclassified images per class.

    Args:
    - misclassified_examples: Dictionary storing misclassified examples {true_label: [(image, true_label, pred_label), ...]}
    - num_examples: Number of examples per class to display
    """
    for true_label, examples in misclassified_examples.items():
        print(f"\nMisclassified Examples for Class {true_label}:")
        num_to_show = min(num_examples, len(examples))

        fig, axes = plt.subplots(1, num_to_show, figsize=(15, 5))
        for i in range(num_to_show):
            img, true_label, pred_label = examples[i]
            img = img.permute(1, 2, 0).numpy()  # Convert tensor image to NumPy format

            ax = axes[i] if num_to_show > 1 else axes
            ax.imshow(img, cmap="gray")
            ax.set_title(f"True: {true_label}, Pred: {pred_label}")
            ax.axis("off")

        plt.show()

# Call function to display misclassified examples
show_misclassified_examples(misclassified_examples)

NameError: name 'misclassified_examples' is not defined